In [0]:

import datetime
from pyspark.sql import types as T
from pyspark.sql import functions as F
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")

from aadatapipelinecore.core.urn import Urn
from aadatapipelinecore.core.pipeline import type_
from applications.common.parser import SqlParser
from applications.common.executor import SqlExecutor
from applications.auto_pipeline.transform import _view
from aadatapipelinecore.core.utils.spark import eject_all_caches
import psycopg2
import datetime
from aadatapipelinecore.core.urn import Urn
from aaplproxy.da.local_sqlrunner import LocalSqlRunner
from aadatapipelinecore.core.utils.module import application_settings
from pyspark.sql import Row
from pyspark.sql.types import *

import aaplproxy


start='2020-04-17'
end='2020-04-18'
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
for days in xrange(date_range.days):
    dates.append(real_date1 + datetime.timedelta(days))
dates.reverse()


class DryRunSqlExecutor(SqlExecutor):
    def _verify_tasks(self):
        pass


def run(spark, raw_data, sql_text, dry_run=True):
    urn = Urn(namespace=raw_data["namespace"])
    source_data_list = raw_data.pop("source")
    raw_data.update(raw_data.pop("options"))
    _view(spark, sql_text, None, source_data_list)
    context = raw_data
    tasks = SqlParser(spark, sql_text, context).parse()
    if dry_run:
        sql_executor = DryRunSqlExecutor
    else:
        sql_executor = SqlExecutor
    sql_executor(urn, spark, tasks, type_.EventType.TRANSFORM, context).run()





def test_download_attribution_db(test_date):
    print 'test_data:' , test_date

    spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution.v3/fact/").where("granularity='daily' and  date in ('{}') ".format(test_date)).createOrReplaceTempView("unified_download_attribution")

    sql_text = """
    
    WITH download_attribution AS (
      SELECT app_id, coalesce(free_app_download, 0 ) as free_app_download, coalesce(paid_app_download, 0 ) as paid_app_download, coalesce(revenue, 0 ) as revenue, device_code, country_code, organic_download_share from unified_download_attribution
    );
    
    WITH caculate_data AS (
        SELECT CAST(ROUND (organic_download_share * (free_app_download+paid_app_download)) AS int) AS est_organic_download, 
        CAST(free_app_download+paid_app_download - ROUND(organic_download_share * (free_app_download+paid_app_download)) AS int) as est_paid_download,
        app_id, 
        free_app_download AS est_free_app_download, 
        paid_app_download AS est_paid_app_download, 
        revenue AS est_revenue, 
        device_code, 
        country_code
        FROM download_attribution
    );
    
    
        -- rank_unified,store_unified
    WITH unified_data_test AS 
    ( 
                    SELECT          store_unified.country_code, 
                                    store_unified.device_code, 
                                    store_unified.free_app_download AS est_free_app_download , 
                                    store_unified.paid_app_download AS est_paid_app_download, 
                                    store_unified.revenue           AS est_revenue, 
                                    store_unified.revenue_iap       AS est_revenue_iap, 
                                    store_unified.revenue_non_iap   AS est_revenue_non_iap, 
                                    rank_unified.category_id, 
                                    rank_unified.app_id, 
                                    rank_unified.free_app_download, 
                                    rank_unified.paid_app_download, 
                                    rank_unified.revenue, 
                                    rank_unified.revenue_iap, 
                                    rank_unified.revenue_non_iap, 
                                    rank_unified.granularity, 
                                    rank_unified.date 
                    FROM            rank_unified 
                    FULL OUTER JOIN store_unified 
                    ON              rank_unified.app_id = store_unified.app_id 
                    AND             rank_unified.country_code = store_unified.country_code 
                    AND             rank_unified.device_code = store_unified.device_code 
                    AND             rank_unified.date = store_unified.date );
    
    
    
    WITH unified_rank_filter_data_free_app_download AS 
    ( 
           SELECT * 
           FROM   unified_data_test 
           WHERE ( ( ( 
                                free_app_download<=1000 
                         AND    country_code!="WW" ) 
                  OR     ( 
                                free_app_download<=4000 
                         AND    country_code=="WW" ) ) 
           OR     ( ( 
                                paid_app_download<=1000 
                         AND    country_code!="WW" ) 
                  OR     ( 
                                paid_app_download<=4000 
                         AND    country_code=="WW" ) ) 
           OR     ( ( 
                                revenue<=1000 
                         AND    country_code!="WW" ) 
                  OR     ( 
                                revenue<=4000 
                         AND    country_code=="WW" ) ) )
           AND    device_code!='ios-all'
    );
    
    
    
           WITH unified_category_filter_data_free_app_download AS 
    ( 
           SELECT * ,
           CASE WHEN (free_app_download > 1000 and country_code !='WW') or (free_app_download > 4000 and country_code =='WW' ) or (free_app_download is null or free_app_download <= 0) Then null else est_free_app_download END as est_free_app_download_category,
           CASE WHEN (paid_app_download > 1000 and country_code !='WW') or (paid_app_download > 4000 and country_code =='WW' ) or (paid_app_download is null or paid_app_download <= 0) Then null else est_paid_app_download END as est_paid_app_download_category,
           CASE WHEN (revenue > 1000 and country_code !='WW') or (revenue > 4000 and country_code =='WW') or (revenue is null or revenue <= 0) Then null else est_revenue  END as est_revenue_category
           FROM   unified_rank_filter_data_free_app_download 


    );    

            WITH download_attribution_join_rank_value AS (
            
            SELECT caculate_data.* FROM caculate_data 
            JOIN unified_category_filter_data_free_app_download
            ON caculate_data.app_id = unified_category_filter_data_free_app_download.app_id
            AND caculate_data.country_code = unified_category_filter_data_free_app_download.country_code
            AND caculate_data.device_code = unified_category_filter_data_free_app_download.device_code

            )
    
    """
    namespace = "aa.store.market-size.v1"
    ingest_msg = {
        "namespace": "aa.store.market-size.v1",
        "job_type": "routine",
        "options": {},
        "source": [
             {
                "data_encoding": "parquet",
                "compression": "gzip",
                "name": "store_unified",
                "path": [
                    "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={}".format(
                        test_date)],
                # "path": est_list,
    
            }, {
                "data_encoding": "parquet",
                "compression": "gzip",
                "name": "rank_unified",
                "path": [
                    "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-rank.v1/fact/granularity=daily/date={}".format(
                        test_date)],
                # "path": rank_list,
    
            }
        ]
    }
    
    run(spark, ingest_msg, sql_text)


    # device_code=["ios-phone","ios-tablet","android-all"]

    def citus_row(test_data):
        def get_data_in_citus(date):
            citus_dsn_ = (
            "dbname='{db}' user='{user}' password='{password}' "
            "host='{host}' port='{port}'".format(
                db="aa_store_db",
                user="citus_bdp_prod_app_int_qa",
                host="10.2.6.141",
                password="wZw8cfBuuklIskVG",
                port=5432
            )
        )
    
            sql = '''SELECT 
                        coalesce(est_organic_download, 0 ) AS est_organic_download,
                        coalesce(est_paid_download, 0 ) AS est_paid_download,
                        app_id,
                        coalesce(est_free_app_download, 0 ) AS est_free_app_download,
                        coalesce(est_paid_app_download, 0 ) AS est_paid_app_download,
                        coalesce(est_revenue, 0 ) AS est_revenue,
                        device_code,
                        country_code
                        FROM store.store_est_fact_v1 
                        WHERE date='{}' 
                     '''.format(date)
        
            db_data = query(citus_dsn_, sql)
            return db_data

        def query(dsn, sql):
            with psycopg2.connect(dsn) as conn:
                conn.autocommit = True
                with conn.cursor() as cur:
                    cur.execute(sql)
                    result = cur.fetchall()
                    conn.commit()
            return result
    
        result = get_data_in_citus(test_data)
    # print result

        return [Row(est_organic_download=r[0], est_paid_download=r[1], app_id=r[2], est_free_app_download=r[3], est_paid_app_download=r[4], est_revenue=r[5], device_code=r[6], country_code=r[7]) for r in result]


    test_rdd = sc.parallelize(map(citus_row, [test_date]), 1).cache()

    import pyspark
    test_rdd.unpersist()
    print test_rdd.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    test_rdd.getStorageLevel()
    print(test_rdd.getStorageLevel())
    # print test_rdd.collect()
    new_rdd = test_rdd.flatMap(lambda x: x)
    # print new_rdd.collect()

    schema=StructType([StructField("app_id", LongType(), True),  
                   StructField("country_code", StringType(), True), 
                   StructField("device_code", StringType(), True), 
                   StructField("est_free_app_download", LongType(), True), 
                   StructField("est_organic_download", LongType(), True), 
                   StructField("est_paid_app_download", LongType(), True),
                   StructField("est_paid_download", LongType(), True),
                   StructField("est_revenue", LongType(), True)
                   ])

    new_rdd.toDF(schema).createOrReplaceTempView("data_from_db")

    spark.sql("select distinct app_id, country_code, device_code, est_free_app_download, est_organic_download, est_paid_app_download, est_paid_download, est_revenue  from data_from_db except all select distinct app_id, country_code, device_code, est_free_app_download, est_organic_download, est_paid_app_download, est_paid_download, est_revenue from download_attribution_join_rank_value ").show()
    spark.sql("select distinct app_id, country_code, device_code, est_free_app_download, est_organic_download, est_paid_app_download, est_paid_download, est_revenue from download_attribution_join_rank_value except all select distinct app_id, country_code, device_code, est_free_app_download, est_organic_download, est_paid_app_download, est_paid_download, est_revenue from data_from_db ").show()

    
# PGPASSWORD='wZw8cfBuuklIskVG' psql -h 10.2.6.141  -U citus_bdp_prod_app_int_qa -d aa_store_db -p 5432 << EOF 

    
sc.parallelize(map(test_download_attribution_db, dates), 1)

In [0]:
%%sh
# aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution-category-load.v3/fact/granularity=daily/date=2020-05-02/device_code=android-all/
# aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution-load.v3/fact/granularity=daily/date=2020-05-02/
# aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution.v3/fact/granularity=daily/date=2020-05-02/device_code=android-all/
# aws s3 ls s3://b2c-mktint-prod-dca-kpi/download_attribution/week_and_month_routine/v1.0.0/WEEK/2018-05-12/
# aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution-load.v3/fact/granularity=daily/date=2018-05-10/
aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution.v4/fact/granularity=daily/date=2020-07-11/ --recursive



In [0]:


spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution.v4/fact/granularity=daily/date=2020-07-03/").show()


In [0]:


spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution-load.v3/fact/granularity=daily/date=2018-05-10/").where("app_id='284882215' and country_code='US'").show()

In [0]:
%%sh
# 2018-02-17, 2018-06-02, 2018-06-30, 2019-04-27, 2020-03-28, 2020-04-04
# aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution.v2/fact/granularity=daily/date=2020-04-04/device_code=android-all/
aws s3 ls s3://b2c-mktint-prod-dca-kpi/download_attribution/week_and_month_routine/v1.0.0/WEEK/2018-02-17/AU/ --recursive
aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date=2018-02-17 --recursive | head -3
aws s3 ls s3://b2b-prod-int-data-pipeline-unified/unified/app-int.download-attribution.v1/metric/month=2018-02/date=2018-02-17/device_id=1001/store_id=1/ --recursive


# aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/download-attribution.v1/fact/month=2020-03/date=2020-03-28/ --recursive
# aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/_obselete/download-attribution.v1/fact/month=2018-02/date=2018-02-17/device_id=1001/store_id=1/  --recursive
# aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/legacy_unified_backup/app-int.download-attribution.v1/fact/month=2018-02/date=2018-02-17/device_id=1001/store_id=1/  --recursive

In [0]:

import datetime

start = "2016-09-03"
end = "2020-04-19"
# end = "2012-05-01"
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
sar_list=list()
for days in xrange(date_range.days):
    dates.append(real_date1 + datetime.timedelta(days))
    if (real_date1 + datetime.timedelta(days)).weekday() == 5:
        temp=list()
        while dates:
            temp.append(dates.pop())
        sar_list.append({real_date1 + datetime.timedelta(days):temp})

test_path=list()
for x in sar_list:
    for key,item in x.items():
        test_path.append((
            ["s3://b2c-mktint-prod-dca-kpi/download_attribution/week_and_month_routine/v1.0.0/WEEK/{}/*/".format(key)] , 
            [i.strftime("%Y-%m-%d") for i in item], 
            # ["s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution.v2/fact/granularity=daily/date={}".format(i) for i in item], 
            ["s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={}".format(i) for i in item ]))
        
for x in test_path:
    print x[1]
# ["s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={}".format(test_date)]

In [0]:

import datetime
from pyspark.sql import types as T
from pyspark.sql import functions as F

from aadatapipelinecore.core.urn import Urn
from aadatapipelinecore.core.pipeline import type_
from applications.common.parser import SqlParser
from applications.common.executor import SqlExecutor
from applications.auto_pipeline.transform import _view
from aadatapipelinecore.core.utils.spark import eject_all_caches

spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy


start = "2016-08-28"
end = "2020-04-11"
# end = "2012-05-01"
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
sar_list=list()
for days in xrange(date_range.days):
    dates.append(real_date1 + datetime.timedelta(days))
    if (real_date1 + datetime.timedelta(days)).weekday() == 5:
        temp=list()
        while dates:
            temp.append(dates.pop())
        sar_list.append({real_date1 + datetime.timedelta(days):temp})

test_path=list()
# for x in sar_list:
#     for key,item in x.items():
#         test_path.append(
#             (
#                 ["s3://b2c-mktint-prod-dca-kpi/download_attribution/week_and_month_routine/v1.0.0/WEEK/{}/*/".format(key)] , 
#                 ["s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution.v2/fact/granularity=daily/date={}".format(i) for i in item], 
#                 ["s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={}".format(i) for i in item ]
#             )
#         )
        
for x in sar_list:
    for key,item in x.items():
        test_path.append(
            (
                ["s3://b2c-mktint-prod-dca-kpi/download_attribution/week_and_month_routine/v1.0.0/WEEK/{}/*/".format(key)] , 
                [i.strftime("%Y-%m-%d") for i in item], 
                ["s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={}".format(i) for i in item ]
            )
        )
# print test_path



class DryRunSqlExecutor(SqlExecutor):
    def _verify_tasks(self):
        pass


def run(spark, raw_data, sql_text, dry_run=True):
    urn = Urn(namespace=raw_data["namespace"])
    source_data_list = raw_data.pop("source")
    raw_data.update(raw_data.pop("options"))
    _view(spark, sql_text, None, source_data_list)
    context = raw_data
    tasks = SqlParser(spark, sql_text, context).parse()
    if dry_run:
        sql_executor = DryRunSqlExecutor
    else:
        sql_executor = SqlExecutor
    sql_executor(urn, spark, tasks, type_.EventType.TRANSFORM, context).run()





def test_download_attribution(test_data):
    # print test_data
    spark.read.option("basePath",
                      "s3://b2c-mktint-prod-dca-kpi/download_attribution/week_and_month_routine/v1.0.0/WEEK/").parquet(
        test_data[0][0]).createOrReplaceTempView(
        "download_attribution")

    # spark.read.option("basePath",
    #                   "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily").parquet(
    #     "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={%s}" % ",".join(test_path[0][1])).createOrReplaceTempView(
    #     "store_unified")

    # spark.read.option("basePath",
    #                   "s3://b2c-mktint-prod-dca-kpi/download_attribution/week_and_month_routine/v1.0.0/WEEK/2016-09-03/").parquet(
    #     test_data[0][0]).createOrReplaceTempView(
    #     "download_attribution")
    
    print test_data[1]
    spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution.v2/fact/").where("granularity='daily' and  date in ('{}') ".format("','".join(
        test_data[1]))).createOrReplaceTempView("test_unified_download_attribution")

    sql_text = """

    WITH download_attribution_1 AS (
    
     select device_code, country_code, device_code, granularity, date, product_id, CAST(est_non_organic_download_share as decimal) as est_non_organic_download_share, 
        case when device_code='android-phone' THEN 'android-all' 
             when device_code='ios-phone' THEN 'ios-tablet' END AS new_device_code from download_attribution
    );

    WITH download_attribution_2
    AS (
        select distinct * from ( select new_device_code as device_code, country_code, granularity, date, product_id, CAST(est_non_organic_download_share as decimal) as est_non_organic_download_share from download_attribution_1 
        union all
        select * from download_attribution ) as t1 where device_code!='android-phone'

    );
    
    WITH union_data AS (
    select *, store_unified.device_code as unified_device_code , store_unified.country_code as unified_country_code
    from store_unified full outer join download_attribution_2
    on store_unified.device_code=download_attribution_2.device_code and
    store_unified.country_code=UPPER(download_attribution_2.country_code) and
    store_unified.app_id=download_attribution_2.product_id
    where est_non_organic_download_share is not null
    );
    
    
    WITH calculate_data_prepare AS (
    select app_id,  coalesce(free_app_download, 0 ) as free_app_download,  coalesce(paid_app_download, 0 )  as paid_app_download,revenue, unified_device_code, unified_country_code, est_non_organic_download_share from union_data
    );
    
    

    
    WITH caculate_data AS (
    select app_id,free_app_download,paid_app_download,revenue, unified_device_code, unified_country_code, est_non_organic_download_share, 
    round (( 1 - est_non_organic_download_share ) *(free_app_download+paid_app_download)) as organic_download, ( free_app_download+paid_app_download - ROUND(( 1- est_non_organic_download_share) * (free_app_download+paid_app_download))) as paid_download    from calculate_data_prepare
    );
    
    
    
    WITH compare_data_raw AS (
    select app_id,free_app_download, paid_app_download, paid_download, organic_download, unified_device_code as device_code, unified_country_code as country_code, (1 - est_non_organic_download_share) AS organic_download_share from caculate_data
    );
    
    
    WITH compare_data_unified AS (
    select app_id,coalesce(free_app_download, 0 ) as free_app_download,  coalesce(paid_app_download, 0 ) as paid_app_download, paid_download, organic_download,  device_code,  country_code, organic_download_share from test_unified_download_attribution
    );
    
    
    WITH compare_data_unified_add_to_est AS (
    select app_id, ( coalesce(free_app_download, 0 )  + coalesce(paid_app_download, 0 ) - (paid_download + organic_download)  ) as diff ,  device_code,  country_code from test_unified_download_attribution
    );
    
    """
    
    # store_unified , rank_unified
    namespace = "aa.store.market-size.v1"
    ingest_msg = {
        "namespace": "aa.store.market-size.v1",
        "job_type": "routine",
        "options": {},
        "source": [
            {
                "data_encoding": "parquet",
                "compression": "gzip",
                "name": "store_unified",
                "path": test_data[2]
            }
            # ,
            # {
            #     "data_encoding": "parquet",
            #     "compression": "gzip",
            #     "name": "test_unified_download_attribution",
            #     "path":test_data[1]
            # }
    
        ]
    }
    
    run(spark, ingest_msg, sql_text)
    
    # spark.sql("select * from download_attribution where product_id=284035177 and country_code='ww'").show()
    # spark.sql("select * from compare_data_unified where device_code='ios-tablet' ").show()
    # spark.sql("select * from compare_data_raw").show()

    # spark.sql("select * from caculate_data where app_id=20600000009072 and unified_country_code='WW' and unified_device_code='android-all'").show()
    spark.sql("select * from compare_data_raw where app_id is not null except all select * from compare_data_unified where app_id is not null").show()
    spark.sql("select * from compare_data_unified except all select * from compare_data_raw").show()
    # spark.sql("sselect count(*) from compare_data_raw where app_id is not null").show()
    # spark.sql("select count(*) from compare_data_unified ").show()

    # spark.sql("select * from compare_data_unified_add_to_est where diff !=0  ").show()
    eject_all_caches(spark)

    
# for x in test_path:
    # print x
    # test_download_attribution(x)

    # WITH caculate_data AS (
    # select app_id,free_app_download,paid_app_download,revenue ,unified_granularity, unified_device_code, unified_country_code, est_non_organic_download_share, 

   
    
sc.parallelize(map(test_download_attribution, test_path), 1)

In [0]:
%%sh

# aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/_obselete/download-attribution.v1/fact/month=
# aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/download-attribution.v1/fact/
# aws s3 ls s3://b2c-mktint-prod-dca-kpi/download_attribution/week_and_month_routine/v1.0.0/WEEK/2018-06-02/  --recursive


# aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date=2018-06-02/ --recursive
# aws s3 ls s3://b2b-prod-int-data-pipeline-unified/unified/app-int.download-attribution.v1/metric/month=2018-06/date=2018-06-02/  --recursive

# aws s3 ls s3://b2b-prod-int-data-pipeline-unified/unified/app-int.download-attribution.v1/metric/month=
aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution.v3/fact/granularity=daily/date=2020-04-01/device_code=ios-phone/
 
 
# aws s3 ls s3://b2c-mktint-prod-dca-kpi/download_attribution/week_and_month_routine/v1.0.0/WEEK/

# echo 'hahaha'
# aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/_obselete/download-attribution.v1/fact/month=2018-06/date=2018-06-02/
# # echo 'hahaha'
# aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/legacy_unified_backup/app-int.download-attribution.v1/fact/
# # echo 'hahaha'
# aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/download-attribution.v1/fact/month=2018-06/date=2018-06-02/
# aws s3 ls s3://b2c-mktint-prod-dca-kpi/download_attribution/week_and_month_routine/v1.0.0/WEEK/2018-06-02/  --recursive



# aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily
 



In [0]:

import datetime
from pyspark.sql import types as T
from pyspark.sql import functions as F

from aadatapipelinecore.core.urn import Urn
from aadatapipelinecore.core.pipeline import type_
from applications.common.parser import SqlParser
from applications.common.executor import SqlExecutor
from applications.auto_pipeline.transform import _view
from aadatapipelinecore.core.utils.spark import eject_all_caches

spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy


start = "2016-08-28"
end = "2020-04-25"
# end = "2012-05-01"
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
sar_list=list()
for days in xrange(date_range.days):
    dates.append(real_date1 + datetime.timedelta(days))
    if (real_date1 + datetime.timedelta(days)).weekday() == 5:
        temp=list()
        while dates:
            temp.append(dates.pop())
        sar_list.append({real_date1 + datetime.timedelta(days):temp})

test_path=list()
# for x in sar_list:
#     for key,item in x.items():
#         test_path.append(
#             (
#                 ["s3://b2c-mktint-prod-dca-kpi/download_attribution/week_and_month_routine/v1.0.0/WEEK/{}/*/".format(key)] , 
#                 ["s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution.v2/fact/granularity=daily/date={}".format(i) for i in item], 
#                 ["s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={}".format(i) for i in item ]
#             )
#         )
        
for x in sar_list:
    for key,item in x.items():
        test_path.append(
            (
                ["s3://b2c-mktint-prod-dca-kpi/download_attribution/week_and_month_routine/v1.0.0/WEEK/{}/*/".format(key)] , 
                [i.strftime("%Y-%m-%d") for i in item], 
                ["s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={}".format(i) for i in item ]
            )
        )
# print test_path



class DryRunSqlExecutor(SqlExecutor):
    def _verify_tasks(self):
        pass


def run(spark, raw_data, sql_text, dry_run=True):
    urn = Urn(namespace=raw_data["namespace"])
    source_data_list = raw_data.pop("source")
    raw_data.update(raw_data.pop("options"))
    _view(spark, sql_text, None, source_data_list)
    context = raw_data
    tasks = SqlParser(spark, sql_text, context).parse()
    if dry_run:
        sql_executor = DryRunSqlExecutor
    else:
        sql_executor = SqlExecutor
    sql_executor(urn, spark, tasks, type_.EventType.TRANSFORM, context).run()





def test_download_attribution(test_data):
    # print test_data
    spark.read.option("basePath",
                      "s3://b2c-mktint-prod-dca-kpi/download_attribution/week_and_month_routine/v1.0.0/WEEK/").parquet(
        test_data[0][0]).createOrReplaceTempView(
        "download_attribution")

    # spark.read.option("basePath",
    #                   "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily").parquet(
    #     "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={%s}" % ",".join(test_path[0][1])).createOrReplaceTempView(
    #     "store_unified")

    # spark.read.option("basePath",
    #                   "s3://b2c-mktint-prod-dca-kpi/download_attribution/week_and_month_routine/v1.0.0/WEEK/2016-09-03/").parquet(
    #     test_data[0][0]).createOrReplaceTempView(
    #     "download_attribution")
    
    print test_data[1]
    spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution.v3/fact/").where("granularity='daily' and  date in ('{}') ".format("','".join(
        test_data[1]))).createOrReplaceTempView("test_unified_download_attribution")

    sql_text = """
    
    WITH download_attribution AS (
        select *, CAST(est_non_organic_download_share as decimal(36,20)) as new_est_non_organic_download_share from download_attribution
    );

    WITH download_attribution_1 AS (
    
     select device_code, country_code, granularity, date, product_id, new_est_non_organic_download_share, 
        case when device_code='android-phone' THEN 'android-all' 
             when device_code='ios-phone' THEN 'ios-tablet' END AS new_device_code from download_attribution
    );

    WITH download_attribution_2
    AS (
        select distinct * from ( select new_device_code as device_code, country_code, granularity, date, product_id, new_est_non_organic_download_share as est_non_organic_download_share from download_attribution_1 
        union all
        select device_code, country_code, granularity, date, product_id, est_non_organic_download_share from download_attribution ) as t1 where device_code!='android-phone'

    );
    
    WITH union_data AS (
    select *, store_unified.device_code as unified_device_code , store_unified.country_code as unified_country_code
    from store_unified full outer join download_attribution_2
    on store_unified.device_code=download_attribution_2.device_code and
    store_unified.country_code=UPPER(download_attribution_2.country_code) and
    store_unified.app_id=download_attribution_2.product_id
    where est_non_organic_download_share is not null
    );
    
    
    WITH calculate_data_prepare AS (
    select app_id, coalesce(free_app_download, 0) as free_app_download,  coalesce(paid_app_download, 0 )  as paid_app_download, revenue, unified_device_code, unified_country_code, est_non_organic_download_share from union_data where not (free_app_download is null and paid_app_download is null) 
    );
    
    

    
    WITH caculate_data AS (
    select app_id,free_app_download,paid_app_download,revenue, unified_device_code, unified_country_code, est_non_organic_download_share from calculate_data_prepare
    );
    
    
    
    WITH compare_data_raw AS (
    select app_id,free_app_download, paid_app_download, unified_device_code as device_code, unified_country_code as country_code, (1 - est_non_organic_download_share) AS organic_download_share from caculate_data
    );
    
    
    WITH compare_data_unified AS (
    select app_id,coalesce(free_app_download, 0 ) as free_app_download,  coalesce(paid_app_download, 0 ) as paid_app_download,  device_code,  country_code, organic_download_share from test_unified_download_attribution
    );
    
    

    """
    
    # store_unified , rank_unified
    namespace = "aa.store.market-size.v1"
    ingest_msg = {
        "namespace": "aa.store.market-size.v1",
        "job_type": "routine",
        "options": {},
        "source": [
            {
                "data_encoding": "parquet",
                "compression": "gzip",
                "name": "store_unified",
                "path": test_data[2]
            }
            # ,
            # {
            #     "data_encoding": "parquet",
            #     "compression": "gzip",
            #     "name": "test_unified_download_attribution",
            #     "path":test_data[1]
            # }
    
        ]
    }
    
    run(spark, ingest_msg, sql_text)
    
    # spark.sql("select * from download_attribution where product_id=284035177 and country_code='ww'").show()
    # spark.sql("select * from compare_data_unified where device_code='ios-tablet' ").show()
    # spark.sql("select * from compare_data_raw").show()

    # spark.sql("select * from caculate_data where app_id=20600000009072 and unified_country_code='WW' and unified_device_code='android-all'").show()
    spark.sql("select * from compare_data_raw where app_id is not null except all select * from compare_data_unified where app_id is not null").show()
    spark.sql("select * from compare_data_unified except all select * from compare_data_raw").show()
    count_1 = spark.sql("select count(*) from compare_data_raw where app_id is not null").take(1)
    count_2 = spark.sql("select count(*) from compare_data_unified ").take(1)
    if count_1[0][0] != count_2[0][0]:
        print 'failed!!!!!!!!!!!!!'

    # spark.sql("select * from compare_data_unified_add_to_est where diff !=0  ").show()
    eject_all_caches(spark)

    

    
sc.parallelize(map(test_download_attribution, test_path), 1)

In [0]:

# spark.read.option("basePath",
#                       "s3://b2c-mktint-prod-dca-kpi/download_attribution/week_and_month_routine/v1.0.0/WEEK/").parquet(
#         "s3://b2c-mktint-prod-dca-kpi/download_attribution/week_and_month_routine/v1.0.0/WEEK/2020-04-18/*/").createOrReplaceTempView(
#         "download_attribution")

# spark.sql("select *, CAST(est_non_organic_download_share as decimal(36,20)) as new_est_non_organic_download_share from download_attribution ").show(20,False)


spark.sql("select * from compare_data_raw where app_id=581983831 and country_code='AU' order by free_app_download desc ").show()
spark.sql("select * from compare_data_unified where app_id=581983831 and country_code='AU' order by free_app_download desc ").show()
spark.sql("select * from union_data where  app_id=581983831 and unified_country_code='AU' order by free_app_download desc ").show()


# spark.sql("select * from compare_data_raw where  country_code='AU' order by free_app_download desc ").show()
# spark.sql("select * from compare_data_unified where country_code='AU' order by free_app_download desc ").show()
spark.sql("select * from union_data where unified_country_code='AU' and free_app_download is null and paid_app_download is not null order by free_app_download desc ").show()


In [0]:

import datetime
from pyspark.sql import types as T
from pyspark.sql import functions as F

from aadatapipelinecore.core.urn import Urn
from aadatapipelinecore.core.pipeline import type_
from applications.common.parser import SqlParser
from applications.common.executor import SqlExecutor
from applications.auto_pipeline.transform import _view
from aadatapipelinecore.core.utils.spark import eject_all_caches

spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy


start = "2016-08-28"
end = "2016-08-05"
# end = "2012-05-01"
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
sar_list=list()
for days in xrange(date_range.days):
    dates.append(real_date1 + datetime.timedelta(days))
    if (real_date1 + datetime.timedelta(days)).weekday() == 5:
        temp=list()
        while dates:
            temp.append(dates.pop())
        sar_list.append({real_date1 + datetime.timedelta(days):temp})

test_path=list()
# for x in sar_list:
#     for key,item in x.items():
#         test_path.append(
#             (
#                 ["s3://b2c-mktint-prod-dca-kpi/download_attribution/week_and_month_routine/v1.0.0/WEEK/{}/*/".format(key)] , 
#                 ["s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution.v2/fact/granularity=daily/date={}".format(i) for i in item], 
#                 ["s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={}".format(i) for i in item ]
#             )
#         )
        
for x in sar_list:
    for key,item in x.items():
        test_path.append(
            (
                ["s3://b2c-mktint-prod-dca-kpi/download_attribution/week_and_month_routine/v1.0.0/WEEK/{}/*/".format(key)] , 
                [i.strftime("%Y-%m-%d") for i in item], 
                ["s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={}".format(i) for i in item ]
            )
        )
# print test_path



class DryRunSqlExecutor(SqlExecutor):
    def _verify_tasks(self):
        pass


def run(spark, raw_data, sql_text, dry_run=True):
    urn = Urn(namespace=raw_data["namespace"])
    source_data_list = raw_data.pop("source")
    raw_data.update(raw_data.pop("options"))
    _view(spark, sql_text, None, source_data_list)
    context = raw_data
    tasks = SqlParser(spark, sql_text, context).parse()
    if dry_run:
        sql_executor = DryRunSqlExecutor
    else:
        sql_executor = SqlExecutor
    sql_executor(urn, spark, tasks, type_.EventType.TRANSFORM, context).run()





def test_download_attribution(test_data):
    # print test_data
    spark.read.option("basePath",
                      "s3://b2c-mktint-prod-dca-kpi/download_attribution/week_and_month_routine/v1.0.0/WEEK/").parquet(
        test_data[0][0]).createOrReplaceTempView(
        "download_attribution")

    # spark.read.option("basePath",
    #                   "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily").parquet(
    #     "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={%s}" % ",".join(test_path[0][1])).createOrReplaceTempView(
    #     "store_unified")

    # spark.read.option("basePath",
    #                   "s3://b2c-mktint-prod-dca-kpi/download_attribution/week_and_month_routine/v1.0.0/WEEK/2016-09-03/").parquet(
    #     test_data[0][0]).createOrReplaceTempView(
    #     "download_attribution")
    
    print test_data[1]
    spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution.v3/fact/").where("granularity='daily' and  date in ('{}') ".format("','".join(
        test_data[1]))).createOrReplaceTempView("test_unified_download_attribution")

    sql_text = """
    
    WITH download_attribution AS (
        select *, CAST(est_non_organic_download_share as decimal(36,20)) as new_est_non_organic_download_share from download_attribution
    );

    WITH download_attribution_1 AS (
    
     select device_code, country_code, granularity, date, product_id, new_est_non_organic_download_share, 
        case when device_code='android-phone' THEN 'android-all' 
             when device_code='ios-phone' THEN 'ios-tablet' END AS new_device_code from download_attribution
    );

    WITH download_attribution_2
    AS (
        select distinct * from ( select new_device_code as device_code, country_code, granularity, date, product_id, new_est_non_organic_download_share as est_non_organic_download_share from download_attribution_1 
        union all
        select device_code, country_code, granularity, date, product_id, est_non_organic_download_share from download_attribution ) as t1 where device_code!='android-phone'

    );
    
    WITH union_data AS (
    select *, store_unified.device_code as unified_device_code , store_unified.country_code as unified_country_code
    from store_unified full outer join download_attribution_2
    on store_unified.device_code=download_attribution_2.device_code and
    store_unified.country_code=UPPER(download_attribution_2.country_code) and
    store_unified.app_id=download_attribution_2.product_id
    where est_non_organic_download_share is not null
    );
    
    
    WITH calculate_data_prepare AS (
    select app_id, coalesce(free_app_download, 0) as free_app_download,  coalesce(paid_app_download, 0 )  as paid_app_download, revenue, unified_device_code, unified_country_code, est_non_organic_download_share from union_data where not (free_app_download is null and paid_app_download is null) 
    );
    
    

    
    WITH caculate_data AS (
    select app_id,free_app_download,paid_app_download,revenue, unified_device_code, unified_country_code, est_non_organic_download_share from calculate_data_prepare
    );
    
    
    
    WITH compare_data_raw AS (
    select app_id,free_app_download, paid_app_download, unified_device_code as device_code, unified_country_code as country_code, (1 - est_non_organic_download_share) AS organic_download_share from caculate_data
    );
    
    
    WITH compare_data_unified AS (
    select app_id,coalesce(free_app_download, 0 ) as free_app_download,  coalesce(paid_app_download, 0 ) as paid_app_download,  device_code,  country_code, organic_download_share from test_unified_download_attribution
    );
    
    

    """
    
    # store_unified , rank_unified
    namespace = "aa.store.market-size.v1"
    ingest_msg = {
        "namespace": "aa.store.market-size.v1",
        "job_type": "routine",
        "options": {},
        "source": [
            {
                "data_encoding": "parquet",
                "compression": "gzip",
                "name": "store_unified",
                "path": test_data[2]
            }
            # ,
            # {
            #     "data_encoding": "parquet",
            #     "compression": "gzip",
            #     "name": "test_unified_download_attribution",
            #     "path":test_data[1]
            # }
    
        ]
    }
    
    run(spark, ingest_msg, sql_text)
    
    # spark.sql("select * from download_attribution where product_id=284035177 and country_code='ww'").show()
    # spark.sql("select * from compare_data_unified where device_code='ios-tablet' ").show()
    # spark.sql("select * from compare_data_raw").show()

    # spark.sql("select * from caculate_data where app_id=20600000009072 and unified_country_code='WW' and unified_device_code='android-all'").show()
    spark.sql("select * from compare_data_raw where app_id is not null except all select * from compare_data_unified where app_id is not null").show()
    spark.sql("select * from compare_data_unified except all select * from compare_data_raw").show()
    count_1 = spark.sql("select count(*) from compare_data_raw where app_id is not null").take(1)
    count_2 = spark.sql("select count(*) from compare_data_unified ").take(1)
    if count_1[0][0] != count_2[0][0]:
        print 'failed!!!!!!!!!!!!!'

    # spark.sql("select * from compare_data_unified_add_to_est where diff !=0  ").show()
    eject_all_caches(spark)

    

    
sc.parallelize(map(test_download_attribution, test_path), 1)

In [0]:

from pyspark.sql.functions import broadcast

spark.sql("select broadcast(*) from compare_data_raw where app_id=1450580739 and country_code='TH' order by free_app_download desc ").explain()
# spark.sql("select * from compare_data_unified where app_id=1450580739 and country_code='TH' order by free_app_download desc ").show()
# spark.sql("select * from union_data where  app_id=1450580739 and unified_country_code='TH' order by free_app_download desc ").show()

# spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/legacy_unified_backup/app-int.download-attribution.v1/fact/month=2018-02/date=2018-02-17/").where("app_id=20600004857029 and store_id=2").show(2)

In [0]:

# ['2017-09-02', '2017-09-01', '2017-08-31', '2017-08-30', '2017-08-29', '2017-08-28', '2017-08-27']
# ['2018-02-17', '2018-02-16', '2018-02-15', '2018-02-14', '2018-02-13', '2018-02-12', '2018-02-11']
# ['2019-04-27', '2019-04-26', '2019-04-25', '2019-04-24', '2019-04-23', '2019-04-22', '2019-04-21']
# ['2020-02-22', '2020-02-21', '2020-02-20', '2020-02-19', '2020-02-18', '2020-02-17', '2020-02-16']
# d1 = ['2020-02-29', '2020-02-28', '2020-02-27', '2020-02-26', '2020-02-25', '2020-02-24', '2020-02-23']
# ['2020-03-07', '2020-03-06', '2020-03-05', '2020-03-04', '2020-03-03', '2020-03-02', '2020-03-01']
# ['2020-03-14', '2020-03-13', '2020-03-12', '2020-03-11', '2020-03-10', '2020-03-09', '2020-03-08']
# ['2020-03-21', '2020-03-20', '2020-03-19', '2020-03-18', '2020-03-17', '2020-03-16', '2020-03-15']
# ['2020-03-28', '2020-03-27', '2020-03-26', '2020-03-25', '2020-03-24', '2020-03-23', '2020-03-22']
d1 = ['2020-04-04', '2020-04-03', '2020-04-02', '2020-04-01', '2020-03-31', '2020-03-30', '2020-03-29']

# d1= ['2018-06-02', '2018-06-01', '2018-05-31', '2018-05-30', '2018-05-29', '2018-05-28', '2018-05-27']
day="2020-04-04"
spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution.v2/fact/").where("granularity='daily' and  date in ('{}') ".format("','".join(d1))).where("app_id=1450580739 and country_code='TH' and date='2020-04-01'  ").orderBy("date").show(200)

# spark.read.option("basePath",
#                       "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily").parquet(
#         "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={%s}"% ','.join(d1)).where("app_id=1450580739 and country_code='TH' and device_code in ('ios-phone', 'ios-tablet') ").orderBy("date").show()
        
# spark.read.option("basePath",
#                       "s3://b2c-mktint-prod-dca-kpi/download_attribution/week_and_month_routine/v1.0.0/WEEK/").parquet(
#         "s3://b2c-mktint-prod-dca-kpi/download_attribution/week_and_month_routine/v1.0.0/WEEK/{}/*".format(day)).where("product_id=1450580739 and country_code='th'").show()
        
spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution.v2/fact/granularity=daily/date=2020-04-01/").where("app_id=1450580739 and country_code='TH' and device_code in ('ios-phone', 'ios-tablet') ").show()


In [0]:

# print download attribution
spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution.v3/fact/").where("granularity='daily' and  date in ('2020-02-22', '2020-02-21', '2020-02-20', '2020-02-19', '2020-02-18', '2020-02-17', '2020-02-16') ").where("app_id=1029094059 and country_code='US' and date='2020-02-16'").show(200)
# est 
# spark.read.option("basePath",
#                       "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily").parquet(
#         "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date=2020-02-{16,17,18,19,20,21,22}").where("app_id=1029094059 and country_code='US' and date='2020-02-16' and device_code='ios-phone'").show()
# # share value
# spark.read.option("basePath",
#                       "s3://b2c-mktint-prod-dca-kpi/download_attribution/week_and_month_routine/v1.0.0/WEEK/").parquet(
#         "s3://b2c-mktint-prod-dca-kpi/download_attribution/week_and_month_routine/v1.0.0/WEEK/2020-02-22/*").where("product_id=1029094059 and country_code='us'").show()

In [0]:

from pyspark.sql import types as T
from pyspark.sql import functions as F
from aadatapipelinecore.core.urn import Urn
from aadatapipelinecore.core.pipeline import type_
from applications.common.parser import SqlParser
from applications.common.executor import SqlExecutor
from applications.auto_pipeline.transform import _view
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy

start = "2018-02-11"
end = "2018-02-18"
# end = "2012-05-01"
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
sar_list=list()
for days in xrange(date_range.days):
    dates.append(real_date1 + datetime.timedelta(days))
    if (real_date1 + datetime.timedelta(days)).weekday() == 5:
        temp=list()
        while dates:
            temp.append(dates.pop())
        sar_list.append({real_date1 + datetime.timedelta(days):temp})

test_path=list()

for x in sar_list:
    for key,item in x.items():
        test_path.append(
            (
                ["s3://b2c-mktint-prod-dca-kpi/download_attribution/week_and_month_routine/v1.0.0/WEEK/{}/*/".format(key)] , 
                [i.strftime("%Y-%m-%d") for i in item], 
                ["s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={}".format(i) for i in item ]
            )
        )
print test_path[0][2]


class DryRunSqlExecutor(SqlExecutor):
    def _verify_tasks(self):
        pass
def run(spark, raw_data, sql_text, dry_run=True):
    urn = Urn(namespace=raw_data["namespace"])
    source_data_list = raw_data.pop("source")
    raw_data.update(raw_data.pop("options"))
    _view(spark, sql_text, None, source_data_list)
    context = raw_data
    tasks = SqlParser(spark, sql_text, context).parse()
    if dry_run:
        sql_executor = DryRunSqlExecutor
    else:
        sql_executor = SqlExecutor
    sql_executor(urn, spark, tasks, type_.EventType.TRANSFORM, context).run()
sql_text ="""

"""
namespace = "aa.store.market-size.v1"
ingest_msg = {
    "namespace": "aa.store.market-size.v1",
    "job_type": "routine",
    "options":{},
    "source": [
        {
            "data_encoding": "parquet",
            "compression": "gzip",
            "name": "store_unified",
            "path": ['s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date=2018-02-13', 's3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date=2018-02-14']
        }
    ]
}
run(spark, ingest_msg, sql_text)
spark.sql("select * from store_unified where app_id=20600004857029 and country_code='CA'").show()

In [0]:
   
ANDROID_CATEGORIES = [
    (1, 400000), (2, 400001), (3, 400022), (4, 400023), (5, 400024),
    (6, 400008), (7, 400011), (8, 400014), (9, 400017), (10, 400020),
    (11, 400025), (12, 400030), (13, 400031), (14, 400032), (15, 400033),
    (16, 400035), (17, 400036), (18, 400038), (19, 400040), (20, 400042),
    (21, 400043), (22, 400044), (23, 400058), (24, 400046), (25, 400047),
    (26, 400048), (27, 400050), (28, 400051), (29, 400052), (30, 400053),
    (31, 400054), (32, 400055), (33, 400056), (34, 400045), (35, 400057),
    (36, 400059), (37, 400060), (38, 400002), (39, 400003), (40, 400021),
    (41, 400004), (42, 400005), (43, 400006), (44, 400007), (46, 400009),
    (47, 400010), (48, 400012), (49, 400013), (51, 400015), (52, 400016),
    (54, 400018), (55, 400019), (56, 400061), (57, 400063), (58, 400064),
    (59, 400065), (60, 400062), (61, 400066), (62, 400067), (63, 400068),
    (64, 400069), (65, 400070), (66, 400026), (67, 400027), (68, 400041),
    (69, 400028), (70, 400029), (71, 400034), (72, 400037), (73, 400039),
    (75, 400049)
]


IOS_CATEGORIES = [
    (36, 100000), (100, 100021), (360, 100030), (361, 100031), (362, 100032),
    (363, 100033), (6000, 100023), (6001, 100077), (6002, 100076), (6003, 100075),
    (6004, 100073), (6005, 100072), (6006, 100070), (6007, 100069), (6008, 100068),
    (6009, 100067), (6010, 100066), (6011, 100065), (6012, 100034), (6013, 100029),
    (6014, 100001), (6015, 100027), (6016, 100026), (6017, 100025), (6018, 100022),
    (6020, 100064), (6021, 100035), (6022, 100024), (6023, 100028), (6024, 100071),
    (6025, 100074), (7001, 100002), (7002, 100003), (7003, 100004), (7004, 100005),
    (7005, 100006), (7006, 100007), (7007, 100008), (7008, 100009), (7009, 100010),
    (7010, 100011), (7011, 100012), (7012, 100013), (7013, 100014), (7014, 100015),
    (7015, 100016), (7016, 100017), (7017, 100018), (7018, 100019), (7019, 100020),
    (13001, 100053), (13002, 100046), (13003, 100049), (13004, 100054), (13005, 100060),
    (13006, 100037), (13007, 100036), (13008, 100038), (13009, 100039), (13010, 100040),
    (13011, 100041), (13012, 100042), (13013, 100043), (13014, 100044), (13015, 100045),
    (13017, 100047), (13018, 100048), (13019, 100050), (13020, 100051), (13021, 100052),
    (13023, 100055), (13024, 100056), (13025, 100057), (13026, 100058), (13027, 100059),
    (13028, 100061), (13029, 100062), (13030, 100063)
]


IOS_STORE_COUNTRY_MAPPING = [
    (0, 'WW'), (143575, 'AL'), (143563, 'DZ'), (143564, 'AO'), (143538, 'AI'),
    (143540, 'AG'), (143505, 'AR'), (143524, 'AM'), (143460, 'AU'), (143445, 'AT'),
    (143568, 'AZ'), (143539, 'BS'), (143559, 'BH'), (143541, 'BB'), (143565, 'BY'),
    (143446, 'BE'), (143555, 'BZ'), (143576, 'BJ'), (143542, 'BM'), (143577, 'BT'),
    (143556, 'BO'), (143525, 'BW'), (143503, 'BR'), (143543, 'VG'), (143560, 'BN'),
    (143526, 'BG'), (143578, 'BF'), (143579, 'KH'), (143455, 'CA'), (143580, 'CV'),
    (143544, 'KY'), (143581, 'TD'), (143483, 'CL'), (143465, 'CN'), (143501, 'CO'),
    (143582, 'CG'), (143495, 'CR'), (143494, 'HR'), (143557, 'CY'), (143489, 'CZ'),
    (143458, 'DK'), (143545, 'DM'), (143508, 'DO'), (143509, 'EC'), (143516, 'EG'),
    (143506, 'SV'), (143518, 'EE'), (143583, 'FJ'), (143447, 'FI'), (143442, 'FR'),
    (143584, 'GM'), (143443, 'DE'), (143573, 'GH'), (143448, 'GR'), (143546, 'GD'),
    (143504, 'GT'), (143585, 'GW'), (143553, 'GY'), (143510, 'HN'), (143463, 'HK'),
    (143482, 'HU'), (143558, 'IS'), (143467, 'IN'), (143476, 'ID'), (143449, 'IE'),
    (143491, 'IL'), (143450, 'IT'), (143511, 'JM'), (143462, 'JP'), (143528, 'JO'),
    (143517, 'KZ'), (143529, 'KE'), (143493, 'KW'), (143586, 'KG'), (143587, 'LA'),
    (143519, 'LV'), (143497, 'LB'), (143588, 'LR'), (143520, 'LT'), (143451, 'LU'),
    (143515, 'MO'), (143530, 'MK'), (143531, 'MG'), (143589, 'MW'), (143473, 'MY'),
    (143532, 'ML'), (143521, 'MT'), (143590, 'MR'), (143533, 'MU'), (143468, 'MX'),
    (143591, 'FM'), (143523, 'MD'), (143592, 'MN'), (143547, 'MS'), (143593, 'MZ'),
    (143594, 'NA'), (143484, 'NP'), (143452, 'NL'), (143461, 'NZ'), (143512, 'NI'),
    (143534, 'NE'), (143561, 'NG'), (143457, 'NO'), (143562, 'OM'), (143477, 'PK'),
    (143595, 'PW'), (143485, 'PA'), (143597, 'PG'), (143513, 'PY'), (143507, 'PE'),
    (143474, 'PH'), (143478, 'PL'), (143453, 'PT'), (143498, 'QA'), (143487, 'RO'),
    (143469, 'RU'), (143598, 'ST'), (143479, 'SA'), (143535, 'SN'), (143599, 'SC'),
    (143600, 'SL'), (143464, 'SG'), (143496, 'SK'), (143499, 'SI'), (143601, 'SB'),
    (143472, 'ZA'), (143466, 'KR'), (143454, 'ES'), (143486, 'LK'), (143548, 'KN'),
    (143549, 'LC'), (143550, 'VC'), (143554, 'SR'), (143602, 'SZ'), (143456, 'SE'),
    (143459, 'CH'), (143470, 'TW'), (143603, 'TJ'), (143572, 'TZ'), (143475, 'TH'),
    (143551, 'TT'), (143536, 'TN'), (143480, 'TR'), (143604, 'TM'), (143552, 'TC'),
    (143537, 'UG'), (143492, 'UA'), (143481, 'AE'), (143444, 'GB'), (143441, 'US'),
    (143514, 'UY'), (143566, 'UZ'), (143502, 'VE'), (143471, 'VN'), (143571, 'YE'),
    (143605, 'ZW')]
    


ANDROID_STORE_COUNTRY_MAPPING = [
    (17, 'AR'), (1, 'AU'), (35, 'AT'), (61, 'AZ'), (11, 'BE'), (18, 'BR'), (47, 'BG'),
    (2, 'CA'), (13, 'CL'), (3, 'CN'), (52, 'CO'), (64, 'CR'), (80, 'HR'), (36, 'CZ'),
    (38, 'DK'), (62, 'EC'), (33, 'EG'), (20, 'FI'), (6, 'FR'), (4, 'DE'), (46, 'GR'),
    (16, 'HK'), (37, 'HU'), (19, 'IN'), (21, 'ID'), (39, 'IE'), (40, 'IL'), (8, 'IT'),
    (9, 'JP'), (53, 'KZ'), (95, 'KE'), (50, 'KW'), (86, 'LV'), (65, 'LB'), (78, 'LT'),
    (24, 'MY'), (26, 'MX'), (23, 'NL'), (41, 'NZ'), (74, 'NG'), (42, 'NO'), (54, 'PK'),
    (56, 'PE'), (31, 'PH'), (28, 'PL'), (43, 'PT'), (84, 'PR'), (73, 'QA'), (44, 'RO'),
    (22, 'RU'), (51, 'SA'), (32, 'SG'), (45, 'SK'), (14, 'ZA'), (27, 'KR'), (5, 'ES'),
    (34, 'SE'), (12, 'CH'), (30, 'TW'), (29, 'TH'), (25, 'TR'), (48, 'UA'), (49, 'AE'),
    (7, 'GB'), (10, 'US'), (15, 'VN'), (1000, 'WW')
]


def device_code_to_feed(market_code, device_code, metric_name):
    mapping = [
        ['apple-store',0,'ios-phone','est_free_app_download'],
        ['apple-store',1,'ios-phone','est_paid_app_download'],
        ['apple-store',2,'ios-phone','est_revenue'],
        ['apple-store',101,'ios-tablet','est_free_app_download'],
        ['apple-store',100,'ios-tablet','est_paid_app_download'],
        ['apple-store',102,'ios-tablet','est_revenue'],
        ['google-play',0,'android-all','est_free_app_download'],
        ['google-play',1,'android-all','est_paid_app_download'],
        ['google-play',2,'android-all','est_revenue'],
    ]
    return [x for x in mapping if (x[0], x[2], x[3]) == (market_code, device_code, metric_name)][0][1]



def country_code_to_id(market_code, code):
    if market_code == 'apple-store':
        ios_mapping = {_code:_id for (_id, _code) in IOS_STORE_COUNTRY_MAPPING}
        return ios_mapping[code]
    else:
        gp_mapping = {_code:_id for (_id, _code) in ANDROID_STORE_COUNTRY_MAPPING}
        return gp_mapping[code]


def category_to_legacy_category(market_code, legacy):
    if market_code == 'apple-store':
        ios_category = {_category:_legacy_category for (_legacy_category,_category) in IOS_CATEGORIES }
        return ios_category[legacy]
    else:
        gp_category =  {_category:_legacy_category for (_legacy_category,_category) in ANDROID_CATEGORIES }
        return gp_category[legacy]


country_code_to_id('apple-store',14380)
# 13028, 100061
# 71, 400034
# category_to_legacy_category("apple-store",100000)


In [0]:

import datetime
from pyspark.sql import types as T
from pyspark.sql import functions as F
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")

from aadatapipelinecore.core.urn import Urn
from aadatapipelinecore.core.pipeline import type_
from applications.common.parser import SqlParser
from applications.common.executor import SqlExecutor
from applications.auto_pipeline.transform import _view
from aadatapipelinecore.core.utils.spark import eject_all_caches
import psycopg2
import datetime
from aadatapipelinecore.core.urn import Urn
from aaplproxy.da.local_sqlrunner import LocalSqlRunner
from aadatapipelinecore.core.utils.module import application_settings
from pyspark.sql import Row
from pyspark.sql.types import *

import aaplproxy


start='2020-04-17'
end='2020-04-18'
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
for days in xrange(date_range.days):
    dates.append(real_date1 + datetime.timedelta(days))
dates.reverse()


class DryRunSqlExecutor(SqlExecutor):
    def _verify_tasks(self):
        pass


def run(spark, raw_data, sql_text, dry_run=True):
    urn = Urn(namespace=raw_data["namespace"])
    source_data_list = raw_data.pop("source")
    raw_data.update(raw_data.pop("options"))
    _view(spark, sql_text, None, source_data_list)
    context = raw_data
    tasks = SqlParser(spark, sql_text, context).parse()
    if dry_run:
        sql_executor = DryRunSqlExecutor
    else:
        sql_executor = SqlExecutor
    sql_executor(urn, spark, tasks, type_.EventType.TRANSFORM, context).run()





def test_download_attribution_db(test_date):
    print 'test_data:' , test_date

    spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution.v3/fact/").where("granularity='daily' and  date in ('{}') ".format(test_date)).createOrReplaceTempView("unified_download_attribution")

    sql_text = """
    
    WITH download_attribution AS (
      SELECT app_id, coalesce(free_app_download, 0 ) as free_app_download, coalesce(paid_app_download, 0 ) as paid_app_download, coalesce(revenue, 0 ) as revenue, device_code, country_code, organic_download_share from unified_download_attribution
    );
    
    WITH caculate_data AS (
        SELECT CAST(ROUND (organic_download_share * (free_app_download+paid_app_download)) AS int) AS est_organic_download, 
        CAST(free_app_download+paid_app_download - ROUND(organic_download_share * (free_app_download+paid_app_download)) AS int) as est_paid_download,
        app_id, 
        free_app_download AS est_free_app_download, 
        paid_app_download AS est_paid_app_download, 
        revenue AS est_revenue, 
        device_code, 
        country_code
        FROM download_attribution
    );
    
    
        -- rank_unified,store_unified
    WITH unified_data_test AS 
    ( 
                    SELECT          store_unified.country_code, 
                                    store_unified.device_code, 
                                    store_unified.free_app_download AS est_free_app_download , 
                                    store_unified.paid_app_download AS est_paid_app_download, 
                                    store_unified.revenue           AS est_revenue, 
                                    store_unified.revenue_iap       AS est_revenue_iap, 
                                    store_unified.revenue_non_iap   AS est_revenue_non_iap, 
                                    rank_unified.category_id, 
                                    rank_unified.app_id, 
                                    rank_unified.free_app_download, 
                                    rank_unified.paid_app_download, 
                                    rank_unified.revenue, 
                                    rank_unified.revenue_iap, 
                                    rank_unified.revenue_non_iap, 
                                    rank_unified.granularity, 
                                    rank_unified.date 
                    FROM            rank_unified 
                    FULL OUTER JOIN store_unified 
                    ON              rank_unified.app_id = store_unified.app_id 
                    AND             rank_unified.country_code = store_unified.country_code 
                    AND             rank_unified.device_code = store_unified.device_code 
                    AND             rank_unified.date = store_unified.date );
    
    
    
    WITH unified_rank_filter_data_free_app_download AS 
    ( 
           SELECT * 
           FROM   unified_data_test 
           WHERE ( ( ( 
                                free_app_download<=1000 
                         AND    country_code!="WW" ) 
                  OR     ( 
                                free_app_download<=4000 
                         AND    country_code=="WW" ) ) 
           OR     ( ( 
                                paid_app_download<=1000 
                         AND    country_code!="WW" ) 
                  OR     ( 
                                paid_app_download<=4000 
                         AND    country_code=="WW" ) ) 
           OR     ( ( 
                                revenue<=1000 
                         AND    country_code!="WW" ) 
                  OR     ( 
                                revenue<=4000 
                         AND    country_code=="WW" ) ) )
           AND    device_code!='ios-all'
    );
    
    
    
           WITH unified_category_filter_data_free_app_download AS 
    ( 
           SELECT * ,
           CASE WHEN (free_app_download > 1000 and country_code !='WW') or (free_app_download > 4000 and country_code =='WW' ) or (free_app_download is null or free_app_download <= 0) Then null else est_free_app_download END as est_free_app_download_category,
           CASE WHEN (paid_app_download > 1000 and country_code !='WW') or (paid_app_download > 4000 and country_code =='WW' ) or (paid_app_download is null or paid_app_download <= 0) Then null else est_paid_app_download END as est_paid_app_download_category,
           CASE WHEN (revenue > 1000 and country_code !='WW') or (revenue > 4000 and country_code =='WW') or (revenue is null or revenue <= 0) Then null else est_revenue  END as est_revenue_category
           FROM   unified_rank_filter_data_free_app_download 


    );    

            WITH download_attribution_join_rank_value AS (
            
            SELECT caculate_data.* FROM caculate_data 
            JOIN unified_category_filter_data_free_app_download
            ON caculate_data.app_id = unified_category_filter_data_free_app_download.app_id
            AND caculate_data.country_code = unified_category_filter_data_free_app_download.country_code
            AND caculate_data.device_code = unified_category_filter_data_free_app_download.device_code

            )
    
    """
    namespace = "aa.store.market-size.v1"
    ingest_msg = {
        "namespace": "aa.store.market-size.v1",
        "job_type": "routine",
        "options": {},
        "source": [
             {
                "data_encoding": "parquet",
                "compression": "gzip",
                "name": "store_unified",
                "path": [
                    "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={}".format(
                        test_date)],
                # "path": est_list,
    
            }, {
                "data_encoding": "parquet",
                "compression": "gzip",
                "name": "rank_unified",
                "path": [
                    "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-rank.v1/fact/granularity=daily/date={}".format(
                        test_date)],
                # "path": rank_list,
    
            }
        ]
    }
    
    run(spark, ingest_msg, sql_text)


    # device_code=["ios-phone","ios-tablet","android-all"]

    def citus_row(test_data):
        def get_data_in_citus(date):
            citus_dsn_ = (
                "dbname='{db}' user='{user}' password='{password}' "
                "host='{host}' port='{port}'".format(
                    db="aa_citus_db",
                    user="citus_bdp_prod_app_int_qa",
                    host="10.2.6.141",
                    password="wZw8cfBuuklIskVG",
                    port=5432
                )
            )
    
            sql = '''SELECT 
                        coalesce(est_organic_download, 0 ) AS est_organic_download,
                        coalesce(est_paid_download, 0 ) AS est_paid_download,
                        app_id,
                        coalesce(est_free_app_download, 0 ) AS est_free_app_download,
                        coalesce(est_paid_app_download, 0 ) AS est_paid_app_download,
                        coalesce(est_revenue, 0 ) AS est_revenue,
                        device_code,
                        country_code
                        FROM store.store_est_fact_v1 
                        WHERE date='{}' 
                     '''.format(date)
        
            db_data = query(citus_dsn_, sql)
            return db_data

        def query(dsn, sql):
            with psycopg2.connect(dsn) as conn:
                conn.autocommit = True
                with conn.cursor() as cur:
                    cur.execute(sql)
                    result = cur.fetchall()
                    conn.commit()
            return result
    
        result = get_data_in_citus(test_data)
    # print result

        return [Row(est_organic_download=r[0], est_paid_download=r[1], app_id=r[2], est_free_app_download=r[3], est_paid_app_download=r[4], est_revenue=r[5], device_code=r[6], country_code=r[7]) for r in result]


    test_rdd = sc.parallelize(map(citus_row, [test_date]), 1).cache()

    import pyspark
    test_rdd.unpersist()
    print test_rdd.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    test_rdd.getStorageLevel()
    print(test_rdd.getStorageLevel())
    # print test_rdd.collect()
    new_rdd = test_rdd.flatMap(lambda x: x)
    # print new_rdd.collect()

    schema=StructType([StructField("app_id", LongType(), True),  
                   StructField("country_code", StringType(), True), 
                   StructField("device_code", StringType(), True), 
                   StructField("est_free_app_download", LongType(), True), 
                   StructField("est_organic_download", LongType(), True), 
                   StructField("est_paid_app_download", LongType(), True),
                   StructField("est_paid_download", LongType(), True),
                   StructField("est_revenue", LongType(), True)
                   ])

    new_rdd.toDF(schema).createOrReplaceTempView("data_from_db")

    spark.sql("select distinct app_id, country_code, device_code, est_free_app_download, est_organic_download, est_paid_app_download, est_paid_download, est_revenue  from data_from_db except all select distinct app_id, country_code, device_code, est_free_app_download, est_organic_download, est_paid_app_download, est_paid_download, est_revenue from download_attribution_join_rank_value ").show()
    spark.sql("select distinct app_id, country_code, device_code, est_free_app_download, est_organic_download, est_paid_app_download, est_paid_download, est_revenue from download_attribution_join_rank_value except all select distinct app_id, country_code, device_code, est_free_app_download, est_organic_download, est_paid_app_download, est_paid_download, est_revenue from data_from_db ").show()

    
# PGPASSWORD='wZw8cfBuuklIskVG' psql -h 10.2.6.141  -U citus_bdp_prod_app_int_qa -d aa_store_db -p 5432 << EOF 

    
sc.parallelize(map(test_download_attribution_db, dates), 1)

In [0]:

import datetime
from pyspark.sql import types as T
from pyspark.sql import functions as F

from aadatapipelinecore.core.urn import Urn
from aadatapipelinecore.core.pipeline import type_
from applications.common.parser import SqlParser
from applications.common.executor import SqlExecutor
from applications.auto_pipeline.transform import _view
from aadatapipelinecore.core.utils.spark import eject_all_caches
import psycopg2
import datetime
import aaplproxy
from aadatapipelinecore.core.urn import Urn
from aaplproxy.da.local_sqlrunner import LocalSqlRunner
from aadatapipelinecore.core.utils.module import application_settings
from pyspark.sql import Row
from pyspark.sql.types import *

spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy

start = '2020-04-17'
end = '2020-04-18'
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
for days in xrange(date_range.days):
    dates.append(real_date1 + datetime.timedelta(days))
dates.reverse()


class DryRunSqlExecutor(SqlExecutor):
    def _verify_tasks(self):
        pass


def run(spark, raw_data, sql_text, dry_run=True):
    urn = Urn(namespace=raw_data["namespace"])
    source_data_list = raw_data.pop("source")
    raw_data.update(raw_data.pop("options"))
    _view(spark, sql_text, None, source_data_list)
    context = raw_data
    tasks = SqlParser(spark, sql_text, context).parse()
    if dry_run:
        sql_executor = DryRunSqlExecutor
    else:
        sql_executor = SqlExecutor
    sql_executor(urn, spark, tasks, type_.EventType.TRANSFORM, context).run()


def test_download_attribution_db(test_date):
    print 'test_data:', test_date

    spark.read.format("delta").load(
        "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution.v3/fact/").where(
        "granularity='daily' and  date in ('{}') ".format(test_date)).createOrReplaceTempView(
        "unified_download_attribution")

    sql_text = """



        -- rank_unified,store_unified
    WITH unified_data_test AS 
    ( 
                    SELECT          store_unified.country_code, 
                                    store_unified.device_code, 
                                    store_unified.free_app_download AS est_free_app_download , 
                                    store_unified.paid_app_download AS est_paid_app_download, 
                                    store_unified.revenue           AS est_revenue, 
                                    store_unified.revenue_iap       AS est_revenue_iap, 
                                    store_unified.revenue_non_iap   AS est_revenue_non_iap, 
                                    rank_unified.category_id, 
                                    rank_unified.app_id, 
                                    rank_unified.free_app_download, 
                                    rank_unified.paid_app_download, 
                                    rank_unified.revenue, 
                                    rank_unified.revenue_iap, 
                                    rank_unified.revenue_non_iap, 
                                    rank_unified.granularity, 
                                    rank_unified.date 
                    FROM            rank_unified 
                    FULL OUTER JOIN store_unified 
                    ON              rank_unified.app_id = store_unified.app_id 
                    AND             rank_unified.country_code = store_unified.country_code 
                    AND             rank_unified.device_code = store_unified.device_code 
                    AND             rank_unified.date = store_unified.date );



    WITH unified_rank_filter_data_free_app_download AS 
    ( 
           SELECT * 
           FROM   unified_data_test 
           WHERE ( ( ( 
                                free_app_download<=1000 
                         AND    country_code!="WW" ) 
                  OR     ( 
                                free_app_download<=4000 
                         AND    country_code=="WW" ) ) 
           OR     ( ( 
                                paid_app_download<=1000 
                         AND    country_code!="WW" ) 
                  OR     ( 
                                paid_app_download<=4000 
                         AND    country_code=="WW" ) ) 
           OR     ( ( 
                                revenue<=1000 
                         AND    country_code!="WW" ) 
                  OR     ( 
                                revenue<=4000 
                         AND    country_code=="WW" ) ) )
           AND    device_code!='ios-all'
    );



           WITH unified_category_filter_data_free_app_download AS 
    ( 
           SELECT * ,
           CASE WHEN (free_app_download > 1000 and country_code !='WW') or (free_app_download > 4000 and country_code =='WW' ) or (free_app_download is null or free_app_download <= 0) Then null else est_free_app_download END as est_free_app_download_category,
           CASE WHEN (paid_app_download > 1000 and country_code !='WW') or (paid_app_download > 4000 and country_code =='WW' ) or (paid_app_download is null or paid_app_download <= 0) Then null else est_paid_app_download END as est_paid_app_download_category,
           CASE WHEN (revenue > 1000 and country_code !='WW') or (revenue > 4000 and country_code =='WW') or (revenue is null or revenue <= 0) Then null else est_revenue  END as est_revenue_category
           FROM   unified_rank_filter_data_free_app_download 


    );    

     WITH download_attribution AS (
      SELECT app_id, coalesce(free_app_download, 0 ) as free_app_download, coalesce(paid_app_download, 0 ) as paid_app_download, coalesce(revenue, 0 ) as revenue, device_code, country_code, organic_download_share from unified_download_attribution
    );
    
    WITH download_attribution_join_rank_value AS (

            SELECT unified_category_filter_data_free_app_download.app_id, 
            unified_category_filter_data_free_app_download.country_code, 
            unified_category_filter_data_free_app_download.device_code, 
            coalesce(unified_category_filter_data_free_app_download.est_free_app_download_category, 0) AS est_free_app_download,
            coalesce(unified_category_filter_data_free_app_download.est_paid_app_download_category, 0) AS est_paid_app_download,
            unified_category_filter_data_free_app_download.est_revenue_category AS est_revenue,
            download_attribution.organic_download_share
            FROM download_attribution 
            JOIN unified_category_filter_data_free_app_download
            ON download_attribution.app_id = unified_category_filter_data_free_app_download.app_id
            AND download_attribution.country_code = unified_category_filter_data_free_app_download.country_code
            AND download_attribution.device_code = unified_category_filter_data_free_app_download.device_code

            );


   

    WITH caculate_data AS (
        SELECT CAST(ROUND (organic_download_share * (est_free_app_download+est_paid_app_download)) AS int) AS est_organic_download, 
        CAST(est_free_app_download+est_paid_app_download - ROUND(organic_download_share * (est_free_app_download+est_paid_app_download)) AS int) as est_paid_download,
        app_id, 
        est_free_app_download, 
        est_paid_app_download, 
        est_revenue, 
        device_code, 
        country_code
        FROM download_attribution_join_rank_value
    );


        
    """
    namespace = "aa.store.market-size.v1"
    ingest_msg = {
        "namespace": "aa.store.market-size.v1",
        "job_type": "routine",
        "options": {},
        "source": [
            {
                "data_encoding": "parquet",
                "compression": "gzip",
                "name": "store_unified",
                "path": [
                    "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={}".format(
                        test_date)],
                # "path": est_list,

            }, {
                "data_encoding": "parquet",
                "compression": "gzip",
                "name": "rank_unified",
                "path": [
                    "s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-rank.v1/fact/granularity=daily/date={}".format(
                        test_date)],
                # "path": rank_list,

            }
        ]
    }

    run(spark, ingest_msg, sql_text)

    # device_code=["ios-phone","ios-tablet","android-all"]

    def citus_row(test_data):
        def get_data_in_citus(date):
            citus_dsn_ = (
                "dbname='{db}' user='{user}' password='{password}' "
                "host='{host}' port='{port}'".format(
                    db="aa_citus_db",
                    user="citus_bdp_usage_qa",
                    host="10.2.10.132",
                    password="dNzWtSV3pKTx",
                    port=5432
                )
            )

            sql = '''SELECT 
                        coalesce(est_organic_download, 0 ) AS est_organic_download,
                        coalesce(est_paid_download, 0 ) AS est_paid_download,
                        app_id,
                        coalesce(est_free_app_download, 0 ) AS est_free_app_download,
                        coalesce(est_paid_app_download, 0 ) AS est_paid_app_download,
                        coalesce(est_revenue, 0 ) AS est_revenue,
                        device_code,
                        country_code
                        FROM store.store_est_category_fact_v6 
                        WHERE date='{}' 
                     '''.format(date)

            db_data = query(citus_dsn_, sql)
            return db_data

        def query(dsn, sql):
            with psycopg2.connect(dsn) as conn:
                conn.autocommit = True
                with conn.cursor() as cur:
                    cur.execute(sql)
                    result = cur.fetchall()
                    conn.commit()
            return result

        result = get_data_in_citus(test_data)
        # print result

        return [
            Row(est_organic_download=r[0], est_paid_download=r[1], app_id=r[2], est_free_app_download=r[3],
                est_paid_app_download=r[4], est_revenue=r[5], device_code=r[6], country_code=r[7]) for r in
            result]

    test_rdd = sc.parallelize(map(citus_row, [test_date]), 1).cache()

    import pyspark
    test_rdd.unpersist()
    print test_rdd.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    test_rdd.getStorageLevel()
    print(test_rdd.getStorageLevel())
    # print test_rdd.collect()
    new_rdd = test_rdd.flatMap(lambda x: x)
    # print new_rdd.collect()

    schema = StructType([StructField("app_id", LongType(), True),
                         StructField("country_code", StringType(), True),
                         StructField("device_code", StringType(), True),
                         StructField("est_free_app_download", LongType(), True),
                         StructField("est_organic_download", LongType(), True),
                         StructField("est_paid_app_download", LongType(), True),
                         StructField("est_paid_download", LongType(), True),
                         StructField("est_revenue", LongType(), True)
                         ])

    new_rdd.toDF(schema).createOrReplaceTempView("data_from_db")

    # spark.sql("select distinct app_id, country_code, device_code, est_free_app_download, est_organic_download, est_paid_app_download, est_paid_download, est_revenue from download_attribution_join_rank_value").show()
    spark.sql(
        "select distinct app_id, country_code, device_code, est_free_app_download, est_organic_download, est_paid_app_download, est_paid_download from data_from_db except all select distinct app_id, country_code, device_code, est_free_app_download, est_organic_download, est_paid_app_download, est_paid_download from caculate_data ").show()
    spark.sql(
        "select distinct app_id, country_code, device_code, est_free_app_download, est_organic_download, est_paid_app_download, est_paid_download from caculate_data except all select distinct app_id, country_code, device_code, est_free_app_download, est_organic_download, est_paid_app_download, est_paid_download from data_from_db ").show()


sc.parallelize(map(test_download_attribution_db, dates), 1)


In [0]:
%%sh

PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -p 7432 -U app_bdp_usage_qa -d dailyest << EOF
--select  app_id,store_id,est_organic_download_share  from plproxy.execute_select_nestloop(\$proxy\$ 
--select  app_id,store_id,est_organic_download_share
--    from da.app_da_daily_estimate limit 5 \$proxy\$) tbl 
--      (app_id BIGINT,store_id INT, est_organic_download_share FLOAT ) order by est_organic_download_share  asc limit 5 ;

select  sum(cnt)   from plproxy.execute_select_nestloop(\$proxy\$ 
select  count(1) as cnt from da.app_da_daily_estimate where date='2016-09-24' \$proxy\$) tbl 
   (cnt bigint);

select  app_id,store_id,est_organic_download_share  from plproxy.execute_select_nestloop(\$proxy\$ 
select  app_id,store_id,est_organic_download_share
    from da.app_da_daily_estimate where app_id=1133281508 and date='2016-09-24' \$proxy\$) tbl 
      (app_id BIGINT,store_id INT, est_organic_download_share FLOAT ) order by est_organic_download_share  asc limit 5 ;



EOF



In [0]:

spark.sql("select * from data_from_db where app_id=1064572106 and country_code='WW' and device_code='ios-phone' ").show()
# spark.sql("select * from unified_rank_filter_data_free_app_download where app_id=1064572106 and country_code='WW' and device_code='ios-phone' ").show()
# spark.sql("select * from download_attribution_join_rank_value where app_id=1064572106 and country_code='WW' and device_code='ios-phone' ").show()
# spark.sql("select * from caculate_data where app_id=1064572106 and country_code='WW' and device_code='ios-phone' ").show()
spark.sql("select count(*) from caculate_data").show()


In [0]:


import psycopg2
import datetime
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy
from aadatapipelinecore.core.urn import Urn
from aaplproxy.da.local_sqlrunner import LocalSqlRunner
from aadatapipelinecore.core.utils.module import application_settings
from pyspark.sql import Row
from pyspark.sql.types import *

# device_code=["ios-phone","ios-tablet","android-all"]

def citus_row(test_data):
    def get_data_in_citus(date):
        citus_dsn_ = (
            "dbname='{db}' user='{user}' password='{password}' "
            "host='{host}' port='{port}'".format(
                db="aa_citus_db",
                user="citus_bdp_usage_qa",
                host="10.2.10.132",
                password="dNzWtSV3pKTx",
                port=5432
            )
        )

        sql = '''SELECT 
                    coalesce(est_organic_download, 0 ) AS est_organic_download,
                    coalesce(est_paid_download, 0 ) AS est_paid_download,
                    app_id,
                    coalesce(est_free_app_download, 0 ) AS est_free_app_download,
                    coalesce(est_paid_app_download, 0 ) AS est_paid_app_download,
                    coalesce(est_revenue, 0 ) AS est_revenue,
                    device_code,
                    country_code
                    FROM store.store_est_fact_v6 
                    WHERE date='{}' 
                 '''.format(date)
        
        db_data = query(citus_dsn_, sql)
        return db_data

    def query(dsn, sql):
        with psycopg2.connect(dsn) as conn:
            conn.autocommit = True
            with conn.cursor() as cur:
                cur.execute(sql)
                result = cur.fetchall()
                conn.commit()
        return result
    
    result = get_data_in_citus(test_data)
    # print result

    return [Row(est_organic_download=r[0], est_paid_download=r[1], app_id=r[2], est_free_app_download=r[3], est_paid_app_download=r[4], est_revenue=r[5], device_code=r[6], country_code=r[7]) for r in result]


test_rdd = sc.parallelize(map(citus_row, ["2020-04-17"]), 1).cache()

import pyspark
test_rdd.unpersist()
print test_rdd.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
test_rdd.getStorageLevel()
print(test_rdd.getStorageLevel())
# print test_rdd.collect()
new_rdd = test_rdd.flatMap(lambda x: x)
# print new_rdd.collect()

schema=StructType([StructField("app_id", LongType(), True),  
                   StructField("country_code", StringType(), True), 
                   StructField("device_code", StringType(), True), 
                   StructField("est_free_app_download", LongType(), True), 
                   StructField("est_organic_download", LongType(), True), 
                   StructField("est_paid_app_download", LongType(), True),
                   StructField("est_paid_download", LongType(), True),
                   StructField("est_revenue", LongType(), True)
                   ])

new_rdd.toDF(schema).createOrReplaceTempView("data_from_db")

spark.sql("select * from data_from_db where  app_id=1064572106 and  country_code='WW' and device_code='ios-phone'").show()
# spark.sql("select app_id, country_code, device_code, est_free_app_download, est_organic_download, est_paid_app_download, est_paid_download, est_revenue  from data_from_db except all select app_id, country_code, device_code, est_free_app_download, est_organic_download, est_paid_app_download, est_paid_download, est_revenue from download_attribution_join_rank_value ").show()
# spark.sql("select distinct app_id, country_code, device_code, est_free_app_download, est_organic_download, est_paid_app_download, est_paid_download, est_revenue from download_attribution_join_rank_value except all select app_id, country_code, device_code, est_free_app_download, est_organic_download, est_paid_app_download, est_paid_download, est_revenue from data_from_db ").show(2000)


In [0]:

spark.sql("select * from data_from_db where app_id=1460772578 and country_code='FI' and device_code='ios-phone' ").show()
spark.sql("select * from download_attribution_join_rank_value where app_id=1460772578 and country_code='FI' and device_code='ios-phone' ").show()

# spark.sql("select * from caculate_data where app_id=1391307262 and country_code='GB' and device_code='ios-phone' except all select * from data_from_db where app_id=1391307262 and country_code='GB' and device_code='ios-phone'").show()



In [0]:
%%sh
PGPASSWORD='dNzWtSV3pKTx' psql -h 10.2.10.132  -U citus_bdp_usage_qa -d aa_citus_db -p 5432 << EOF 
set search_path=store;

select count(*) from (select app_id, category_id, country_code, device_code, est_free_app_download, est_paid_app_download, est_revenue, est_organic_download , est_paid_download, date from store.store_est_category_fact_v6 where granularity='daily' and date in ('2020-04-18') ) as t ;
-- select * from store.store_est_fact_v6 where est_free_app_download is null or est_paid_app_download is null  and granularity='daily' and date='2020-04-17' ;

EOF


In [0]:
%%sh
